In [41]:
import os
import tweepy
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [42]:
consumer_key = os.environ["TWITTER_API_KEY"]
consumer_secret = os.environ["TWITTER_API_KEY_SECRET"]
access_token = os.environ["TWITTER_ACCESS_TOKEN"]
access_token_secret = os.environ["TWITTER_ACCESS_TOKEN_SECRET"]

In [43]:
auth = tweepy.OAuth1UserHandler(
    consumer_key, 
    consumer_secret,
    access_token,
    access_token_secret
    )

api = tweepy.API(auth)
api


In [44]:
me = api.verify_credentials()


In [45]:
search_country = 'India'

In [46]:
def scrape_tweets(search_words_list, date_since, date_until, num_tweets, num_runs, search_country):
    
    tweets_df = pd.DataFrame(columns = ['user_id', 'tweet_id', 'username', 'location', 
                                        'following', 'followers', 'twt_created_at', 'total_tweets', 
                                        'retweet_count', 'text', 'hashtags', 'mentions'])
    
    program_start = time.time()
    
    places = api.search_geo(query=search_country, granularity="country")
    place_id = places[0].id
    count_num = 0
    count_tweet = 0
    for search_words in search_words_list:
        
        for i in range(0, num_runs):
            count_num+=1
            no_tweet = 0
            
            start_run = time.time()
            
            tweets = tweepy.Cursor(api.search_tweets, q=search_words + ' AND ' + ("place:%s" % place_id), lang='en', tweet_mode='extended').items(num_tweets)
            print("count_num = ", count_num)
            
        
            
            tweet_list = [tweet for tweet in tweets]
            print(len(tweet_list))
            # print(tweet_list)
            for tweet in tweet_list:
                count_tweet+=1
                if tweet.created_at.date() <= datetime.strptime(date_until, '%Y-%m-%d').date():
                    
                    user_id = tweet.user.id
                    tweet_id = tweet.id
                    username = tweet.user.screen_name
                    location = tweet.user.location
                    following = tweet.user.friends_count
                    followers = tweet.user.followers_count
                    twt_created_at = tweet.created_at
                    total_tweets = tweet.user.statuses_count
                    retweet_count = tweet.retweet_count
                    hashtags = tweet.entities['hashtags']
                    mentions = tweet.entities['user_mentions']
                
                    try:    
                        text = tweet.retweeted_status.full_text
                    except AttributeError:
                        text = tweet.full_text
                    # print(text)
                    ith_tweet = [user_id, tweet_id, username, location, 
                                following, followers, twt_created_at ,total_tweets, 
                                retweet_count, text, hashtags, mentions]
                    
                    tweets_df.loc[len(tweets_df)] = ith_tweet
            
                    no_tweet += 1
            print("count_tweet = ", count_tweet)
            time.sleep(920)

            end_run = time.time()
            duration_run = round((end_run-start_run)/60, 2)
            
            print('Number of tweets scraped for run {} is {}'.format(i+1, no_tweet))
            print('Time take for {} run to complete is {} mins'.format(i+1, duration_run))
    
    
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
    
    path = os.getcwd()
    filename = path + '/data/' + to_csv_timestamp + '_tweets.csv'
    tweets_df.to_csv(filename, index=False)
    
    program_end = time.time()
    print('Scraping is completed.')    
    print('Total time taken to scrape is {} minutes.'.format(round(program_end-program_start)/60, 2))
    
    return tweets_df
    


In [47]:
# search_queries = ["hygienic OR medical OR lack OR facilities OR beds OR medicines OR hygienic OR hygiene OR #emergency OR #complaints OR #medicalnegligence OR #patientdeath OR @MoHFW_INDIA OR @PMOIndia OR @ArvindKejriwal OR @mansukhmandviya OR @ANI OR @PTI_News OR @narendramodi"] 

In [49]:
search_words_list = ["#malpractice OR #medicalcare OR #patientdeath OR #unionhealthministry OR #hospital OR @MoHFW_INDIA OR @PMOIndia OR @ArvindKejriwal OR @mansukhmandviya OR @ANI OR @PTI_News OR @narendramodi",
                    "beds OR vaccines OR streetlights OR #mcdcomplaint OR #noaction OR #mcd311 OR @MCD_Delhi OR @LtGovDelhi OR @ArvindKejriwal OR @DCkpzon OR @HMOIndia OR @AshwaniKumar_92 OR @narendramodi OR @PMOIndia OR @DelhiComplaint OR @SBM_MCD OR medical+protest OR #MedTwitter"]
date_since = '2020-01-01'
date_until = '2022-11-08'
num_tweets = 2000
num_runs = 2
# time.sleep(920)
scrape_tweets(search_words_list, date_since, date_until, num_tweets, num_runs, search_country)

count_num =  1
2000
count_tweet =  2000
Number of tweets scraped for run 1 is 2000
Time take for 1 run to complete is 18.57 mins
count_num =  2
2000
count_tweet =  4000
Number of tweets scraped for run 2 is 2000
Time take for 2 run to complete is 19.2 mins
count_num =  3
2000
count_tweet =  6000
Number of tweets scraped for run 1 is 2000
Time take for 1 run to complete is 18.15 mins
count_num =  4
2000
count_tweet =  8000
Number of tweets scraped for run 2 is 2000
Time take for 2 run to complete is 19.72 mins
Scraping is completed.
Total time taken to scrape is 75.66666666666667 minutes.


,user_id,tweet_id,username,location,following,followers,twt_created_at,total_tweets,retweet_count,text,hashtags,mentions
0,73341971,1589272727340601347,ravi4354,"India,Not in Hindu Rashtra",634,6419,2022-11-06 15:05:20+00:00,41202,0,This is the Goodwill earned by Mr @narendramod...,[],"[{'screen_name': 'narendramodi', 'name': 'Nare..."
1,704630589110898689,1589272611695259651,iamanshulthakur,"सिवनी,मध्यप्रदेश,भारत",396,2386,2022-11-06 15:04:52+00:00,9400,0,@PrajwalBusta @narendramodi It's very special ...,[],"[{'screen_name': 'PrajwalBusta', 'name': 'Praj..."
2,1113372979709108224,1589272584293863427,LifestyleVishnu,"Ambikapur, India",304,286,2022-11-06 15:04:46+00:00,5764,0,@AjayHimatlal @YssSpeaks @BJP4Haryana @cmohry ...,"[{'text': 'NoActionOnArrestWarrant', 'indices'...","[{'screen_name': 'AjayHimatlal', 'name': 'Ajay..."
3,2249900360,1589272560122073088,prkgarg,GLOBE,709,203,2022-11-06 15:04:40+00:00,24799,0,@ArvindKejriwal You will not get more than 2 %...,[],"[{'screen_name': 'ArvindKejriwal', 'name': 'Ar..."
4,780816899072331776,1589272554308796417,SunilBhatM,"New Delhi, India",930,2284,2022-11-06 15:04:38+00:00,8235,0,Magnificent!\n\nI visited Pradhan Manthri Sang...,[],"[{'screen_name': 'narendramodi', 'name': 'Nare..."
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,1378678156161249282,1588512517491666951,vimal_mandal21,"New Delhi, India",25,6,2022-11-04 12:44:31+00:00,87,0,@MCD_Delhi @rameshbidhuri @RamSinghNetaji @Arv...,[],"[{'screen_name': 'MCD_Delhi', 'name': 'Municip..."
7996,1280873038133604362,1588512273303474176,SheikhHamidRza,"छत्तीसगढ़, भारत",29,14,2022-11-04 12:43:33+00:00,81,2,PRE REPUBLIC DAY CAMP (COUNTDOWN STARTED)@nssg...,[],"[{'screen_name': 'nssggcuCG', 'name': 'NSS Gur..."
7997,451076595,1588511889767936000,vinayfast,Mumbai,405,77,2022-11-04 12:42:02+00:00,671,1,@BJP4India @narendramodi Long live Modi ji \n\...,[],"[{'screen_name': 'BJP4India', 'name': 'BJP', '..."
7998,1099949377056051200,1588511676370157574,RishiGupta_JNU,"New Delhi, India",440,214,2022-11-04 12:41:11+00:00,334,0,"@ArvindKejriwal dear Mr. CM, your good governa...","[{'text': 'Delhi', 'indices': [99, 105]}, {'te...","[{'screen_name': 'ArvindKejriwal', 'name': 'Ar..."
